In [ ]:
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *
import datetime
import os
import pickle

# history
按照是否高峰、是否工作日、是否下雨统计历史平均距离和速度，结果保存在./history下

In [1]:
def find_history2(is_rain,is_workday,is_peek,buss,busv):
    #buss = buss[(buss['Distance'] < 10) & (buss['Diff_Time'] < 1000)]
    if is_rain ==0 and is_workday ==1 :
        days = ['02','03','04','05','06','11','12','13','16','19','20','23','24','25','26','27','30','31']
    elif is_rain ==0 and is_workday ==0 :
        days = ['01','15','22','28','19']
    elif is_rain ==1 and is_workday ==1 :
        days = ['09','10','17','18']
    else :
        days = ['07','08','14','21']
    if is_peek == 1:
        time_hour = [('07','09'),('16','19')]
    else :
        time_hour = [('06','07'),('09','16'),('19','23')]
    bus = pd.DataFrame()
#     print(len(days))
    for i in range(len(days)):
#         d11 = datetime.datetime.now()
        day = days[i]
        for j in range(len(time_hour)):
            start = '2017-10-' + day + ' ' + time_hour[j][0] + ':00:00'
            end =  '2017-10-' + day + ' ' + time_hour[j][1] + ':00:00'
                
            index = (busv['O_TIME']>=start)&(busv['O_TIME']<end)#&(busv['O_UP']==on)&(busv['Source_Station']==start_station)&(busv['Target_Station']==next_station)
#             print(start,end)
            b = buss[index]
            
#             b['O_TIME'] = busv[index]['O_TIME']
            bus = pd.concat([bus,b])
#         d3 = datetime.datetime.now()
#         print('time2',(d3 - d11).microseconds)
    return bus

def history(num):
    d1 =  datetime.datetime.now()
    busV = pd.DataFrame()
    busS = pd.DataFrame()
    for d in range(9, 32):
        if d < 10:
            day = '0' + str(d)
        else:
            day = str(d)
        path1 = './' + day + '/' + str(num) +'-v.txt';
        path2 = './' + day + '/' + str(num) +'-station.txt';
        if os.path.exists(path1) and os.path.exists(path2):
#             print(path1,path2)#./24/621-v.txt ./24/621-station.txt
            busv = pd.read_table(path1,header=None,sep=',',encoding='utf-8')
            if d >= 25 :
                busv.columns=['O_TERMINALNO','O_TIME',"O_UP","Source_Station","Target_Station","MaxV"]
            else :
                busv.columns=['O_TIME',"O_UP","Source_Station","Target_Station","MaxV"]
            buss = pd.read_table(path2,header=None,sep=',',encoding='utf-8')
            buss.columns=["O_UP","Source_Station","Target_Station","Distance",'Diff_Time']
            buss['O_TIME'] = busv['O_TIME']
            busV = pd.concat([busV,busv])
            busS = pd.concat([busS,buss])
    
    if (busV.shape[0]>0 and busS.shape[0]>0):
        for is_rain in range(0,2):
            for is_work in range(0,2):
                for is_peek in range(0,2):
                    b = find_history2(is_rain,is_work,is_peek,busS,busV)
                    b = b[(b['Distance'] < 10) & (b['Diff_Time'] < 1000)]
#                     b = b.groupby(['O_UP','Source_Station','Target_Station']).mean()
                    with open('./history/'+str(num)+'_'+str(is_rain)+'_'+str(is_work)+'_'+str(is_peek)+'.txt', 'wb') as data_file:
                        pickle.dump(b, data_file)
    d2 =  datetime.datetime.now()
    print('time',(d2 - d1).microseconds)

In [ ]:
#history
# line = pd.DataFrame()
# for d in range(25, 32):
#     if d < 10:
#         day = '0' + str(d)
#     else:
#         day = str(d)

#     train = pd.read_csv("./data/train201710"+day+".csv", sep=",")
#     train['O_TIME'] = pd.to_datetime('2017-10-' + day + ' ' + train['O_TIME'], format='%Y-%m-%d %H:%M:%S')
#     train = train.groupby(['O_LINENO', 'O_TERMINALNO']).apply(lambda x: x.sort_values(by='O_TIME', ascending=True))
    
#     line = pd.concat([line,train['O_LINENO']])
# line = line.drop_duplicates()
# with open('./history/line_test.txt', 'wb') as data_file:
#     pickle.dump(line, data_file)
with open('./data/line_test.txt', 'rb') as data_file:
    line = pickle.load(data_file)
with open('./data/line_test.txt', 'rb') as data_file:
    line1 = pickle.load(data_file)
    line = pd.concat([line,line1])
line = line.drop_duplicates()
line.columns=['line']
print(line.shape)
for i in range(line.shape[0]):
    l = int(line['line'].iloc[i])
    history(l)
    print(l,"finished")